In [24]:
%%capture output

def compute_non_trivial(adj_matrix):
    result = []
    for i in range(adj_matrix.shape[0]):
        if adj_matrix[i].any() or adj_matrix[:,i].any():
            result.append(i)
    return result

from utils import data
import uclasm
import time
import matplotlib.pyplot as plt
import equivalence_partition

tmplts, world = data.pnnl_v6(5)
#tmplts, world = data.gordian_v7(2)
#tmplts, world = data.pnnl_rw_noisy()
#tmplts, world = data.pnnl_v6(7)
#tmplts, world = data.pnnl_rw()
#tmplts, world = data.ivysys_v7()


print("Loading data...")
st = time.time()
tmplt  = tmplts[0]
print("Took %f seconds to load data"%(time.time()-st))

results = {} # stores 'ch': partition
time_mat = {} # stores 'ch' : time to compute partition

#### set this graph to run experiment
graph = tmplt
####################################

print("Size of graph: " + str(graph.n_nodes))

# to compute non-trivial
non_triv = {}

for ch, sparse_mat in graph.ch_to_adj.items():
    # preprocessing
    adj_matrix = sparse_mat.toarray() # this might be too big for certain dataset
    # compute non-triv
    non_triv[ch] = compute_non_trivial(adj_matrix)
    # compute equivalence classes
    start_time = time.time()
    results[ch] = equivalence_partition.partition_vertices(adj_matrix)
    time_mat[ch] = time.time() - start_time # save the time took to compute the partition
    print("Finished channel: "+ ch + ". Took %5f seconds"%(time_mat[ch]))

equivpartition = equivalence_partition.combine_channel_equivalence(results)

In [25]:
print_stats(equivpartition)

==== GENERAL-STATS =====
Total number of original nodes: 64
Total number of equiv classes: 57
Compression percentage (1 - equiv_classes/total_number_of_nodes): 0.11
===== EQUIVALENCE CLASSES ===== 
[{0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}, {10}, {11}, {12}, {13}, {14}, {15}, {16}, {17}, {18}, {19}, {20}, {21}, {22}, {23}, {24}, {25}, {26}, {27}, {28}, {29}, {30}, {31}, {32}, {33}, {49, 34}, {35}, {51, 36, 37, 38}, {39}, {40}, {41}, {42}, {43}, {44}, {45}, {46}, {47}, {48}, {50}, {52}, {53}, {54}, {55}, {56}, {57}, {58}, {59}, {60, 61, 62, 63}]
===== NON-TRIVIAL EQUIV CLASSES ===== 
[{49, 34}, {51, 36, 37, 38}, {60, 61, 62, 63}]
Number of non-trivial equiv classes: 3
Sizes of equiv classes: [2, 4, 4]
Largest equiv size: 4


In [6]:
verbose = True
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
def print_stats(partition, detailed=True):
    equiv_classes = partition.classes()
    # compute some stats
    print("==== GENERAL-STATS =====")
    # the number of nodes
    print("Total number of original nodes: " + str(len(partition))    )  
    # the number of equiv classes
    print("Total number of equiv classes: " + str(len(equiv_classes))) 
    # compression percentage
    compression_percentage = 1- len(equiv_classes)/len(partition)
    print("Compression percentage (1 - equiv_classes/total_number_of_nodes): %4.2f"%(compression_percentage))
    equiv_classes = partition.classes() 
    
    if detailed:
        print("===== EQUIVALENCE CLASSES ===== ")
        print(equiv_classes)
    
    print("===== NON-TRIVIAL EQUIV CLASSES ===== ")
    non_triv = [i for i in equiv_classes if len(i) > 1]
    if detailed:
        print(non_triv)
    print("Number of non-trivial equiv classes: %d"%len(non_triv))
    
    sizes = [len(l) for l in non_triv]
    if detailed:
        print("Sizes of equiv classes: "+ str(sizes))
        print("Largest equiv size: %d"%(max(sizes)))



=========STATS FOR CHANNEL Comm==============
--> Time took: 0.010499
==== GENERAL-STATS =====
Total number of original nodes: 92
Total number of equiv classes: 25
Compression percentage (1 - equiv_classes/total_number_of_nodes): 0.73
===== EQUIVALENCE CLASSES ===== 
[{0}, {1}, {2}, {3}, {4, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90}, {5}, {6}, {7}, {8}, {9}, {10}, {11}, {28}, {29}, {30}, {31}, {56}, {57, 58, 60, 61}, {59}, {62}, {65, 63}, {64}, {66}, {75}, {91}]
===== NON-TRIVIAL EQUIV CLASSES ===== 
[{4, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90}, {57, 58, 60, 61}, {65, 63

In [9]:
for ch in results:
    print("\n" + bcolors.WARNING+\
          "=========STATS FOR CHANNEL "+ ch+ "==============" + bcolors.ENDC)
    print("--> Time took: %f"%(time_mat[ch]))
    print_stats(results[ch], detailed=verbose)


=========STATS FOR CHANNEL Comm==============
--> Time took: 0.010499
==== GENERAL-STATS =====
Total number of original nodes: 92
Total number of equiv classes: 25
Compression percentage (1 - equiv_classes/total_number_of_nodes): 0.73
===== EQUIVALENCE CLASSES ===== 
[{0}, {1}, {2}, {3}, {4, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90}, {5}, {6}, {7}, {8}, {9}, {10}, {11}, {28}, {29}, {30}, {31}, {56}, {57, 58, 60, 61}, {59}, {62}, {65, 63}, {64}, {66}, {75}, {91}]
===== NON-TRIVIAL EQUIV CLASSES ===== 
[{4, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90}, {57, 58, 60, 61}, {65, 63

In [15]:
output.show()

Loading data...
Took 0.000096 seconds to load data
Size of graph: 156
Finished channel: 0. Took 0.113416 seconds
Finished channel: 10. Took 0.008129 seconds
Finished channel: 1. Took 0.054161 seconds
Finished channel: 2. Took 0.016850 seconds
Finished channel: 3. Took 0.009246 seconds
Finished channel: 4. Took 0.007632 seconds
Finished channel: 5. Took 0.024139 seconds
Finished channel: 6. Took 0.007635 seconds
Finished channel: 8. Took 0.013842 seconds
Finished channel: 9. Took 0.008142 seconds
